### Imports

In [1]:
import numpy as np
import tqdm
import matplotlib.pyplot as plt
# from jaxtyping import Float

### Problem Statement

A big group of Sea Turtles is visiting our shores, bringing with them an opportunity to acquire some top grade `FLIPPERS`. You only have two chances to offer a good price. Each one of the Sea Turtles will accept the lowest bid that is over their reserve price. 

The distribution of reserve prices is uniform between 160–200 and 250–320, but none of the Sea Turtles will trade between 200 and 250 due to some ancient superstition.

For your second bid, they also take into account the average of the second bids by other traders in the archipelago. They’ll trade with you when your offer is above the average of all second bids. But if you end up under the average, the probability of a deal decreases rapidly. 

To simulate this probability, the PNL obtained from trading with a fish for which your second bid is under the average of all second bids will be scaled by a factor *p*:

$$
p = \left(\frac{320 – \text{average bid}}{320 – \text{your bid}}\right)^3
$$

You know there’s a constant desire for Flippers on the archipelago. So, at the end of the round, you’ll be able to sell them for 320 SeaShells ****a piece.

Think hard about how you want to set your two bids, place your feet firmly in the sand and brace yourself, because this could get messy.

### Solution

First, we determine the probability multiplier for below average second bids.

In [2]:
def second_bid_scaler(
    bid: int,
    average_second_bid: int = 320,
    optimal_opponents: bool = True,
) -> float:
    """
    This function calculates the second bid scaling factor based on the average second bid and the number of opponents.
    
    :param average_second_bid: The average second bid amount.
    :param optimal_opponents: A boolean indicating whether the opponents are optimal.
    :return: The scaling factor for the second bid.
    """
    if optimal_opponents:
        return 1.0
    else:
        if bid >= average_second_bid:
            return 1.0
        else:
            return np.power((320 - average_second_bid) / (320 - bid), 3)

To begin with, we optimize assuming all other agents also choose the optimal second bid, i.e. our second bid is exactly the average and everyone has the same second bid. In this case, given $n$ turtles to trade with, each with reserve price $R_1, \ldots, R_n\overset{\text{iid}}{\sim} \text{Unif}([160, 200] \cup [250, 320])$. Let our first and second bids be $p_1$ and $p_2$, respectively. We seek to find a solution to the following objective
$$\max_{p_1, p_2} \sum_{i=1}^n \left[(320 - p_1) \cdot 1_{R_i\leq p_1} + (320-p_2) \cdot 1_{p_1 < R_i \leq p_2}\right].$$ 
Since $n$ is large, the above is equivalent to
\begin{align*}
    \max_{p_1, p_2} &\left[(320 - p_1) \mathbb{P}(R \leq p_1) + (320-p_2)\mathbb{P}(p_1 < R\leq p_2)\right], \\
    \text{subject to } & 160 \leq p_1 \leq p_2 \leq 320, \\
    & p_1, p_2\in \mathbb{N}.
\end{align*}
The pdf of $R$ is given by
$$f_R(r) = \begin{cases} 1/110 & \text{ if } r\in [160, 200] \cup [250, 320], \\ 0 & \text{ otherwise.}\end{cases}$$
We may then rewrite our objective function as
$$\max_{p_1, p_2} \left[(320 - p_1)\int_{160}^{p_1} f_R(r) dr + (320-p_2)\int_{p_1}^{p_2} f_R(r)dr\right].$$
We proceed by testing every possible pair $(p_1, p_2)$ satisfying the constraints by Monte Carlo and analytic methods.

In [3]:
def objective_fn(
    bid1: int,
    bid2: int,
    monte_carlo: bool,
    num_simulations: int = 500000,
) -> float:
    """
    This function calculates the objective function value based on the bids.
    
    :param bid1: The first bid amount.
    :param bid2: The second bid amount.
    :return: The objective function value.
    """

    if monte_carlo:
        probs = np.random.rand(num_simulations)

        r1 = np.random.uniform(160, 200, size=num_simulations)
        r2 = np.random.uniform(250, 320, size=num_simulations)

        p = 40 / 110

        reserves = np.where(probs < p, r1, r2)
        objectives = np.where(bid1 >= reserves, 320 - bid1,
                    np.where(bid2 >= reserves, 320 - bid2, 0))
        
        return np.mean(objectives)
    
    else:

        if bid1 <= 200 and bid2 < 200:
            return (320 - bid1) * (bid1 - 160) / 110 + (320 - bid2) * (bid2 - bid1) / 110
        elif bid1 <= 200 and bid2 >= 250:
            return (320 - bid1) * (bid1 - 160) / 110 + (320 - bid2) * (bid2 - bid1 - 50) / 110
        elif bid1 >= 250 and bid2 >= 250:
            return (320 - bid1) * (bid1 - 210) / 110 + (320 - bid2) * (bid2 - bid1) / 110
        else:
            return 0


def get_optimal_bids(monte_carlo: bool = True):
    """
    This function calculates the optimal bids for the first and second bidders.
    
    :return: A tuple containing the optimal bids for the first and second bidders.
    """

    argmax = []
    objective_opt = -np.inf

    pbar = tqdm.tqdm(range(160, 321))

    for bid1 in pbar:
        for bid2 in range(bid1, 321):
            objective = objective_fn(bid1, bid2, monte_carlo=monte_carlo)
            # if np.isclose(objective, objective_opt):
            #     argmax.append((bid1, bid2))
            # elif objective > objective_opt:
            #     objective_opt = objective
            #     argmax = [(bid1, bid2)]

            if objective > objective_opt:
                objective_opt = objective
                argmax = [(bid1, bid2)]
                
    return argmax, objective_opt

#### Monte Carlo

In [4]:
np.random.seed(0)
argmax, objective_opt = get_optimal_bids(monte_carlo=True)
print(f"Optimal bids: {argmax}, Objective value: {objective_opt}")

100%|██████████| 161/161 [02:01<00:00,  1.33it/s]

Optimal bids: [(200, 287)], Objective value: 54.848892


#### Analytic

In [5]:
argmax, objective_opt = get_optimal_bids(monte_carlo=False)
print(f"Optimal bids: {argmax}, Objective value: {objective_opt}")

100%|██████████| 161/161 [00:00<00:00, 80180.83it/s]

Optimal bids: [(200, 285)], Objective value: 54.772727272727266
